In [81]:
# Import general libraries
import geopandas as gpd

# Import Bokeh libraries
from bokeh.tile_providers import get_provider, Vendors
import json
from bokeh.io import show
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure,output_file, save


#### Read files and set coordinate reference system

In [82]:
# Read files
population = gpd.read_file("population_data/population.shp")
stations_population = gpd.read_file("population_data/stations_population.shp")
stations = gpd.read_file("stations_data/stations.shp")
rails = gpd.read_file("railway_data/railways.shp")

# Set crs to EPSG:3857
population = population.to_crs(epsg=3857)
stations_population = stations_population.to_crs(epsg=3857)
stations = stations.to_crs(epsg=3857)
rails = rails.to_crs(epsg=3857)

#### Convert rails geoDataFrame to ColumnDataSource

In [83]:
# Create function getlinecoords
def getLineCoords(row, geom, coord_type):
    """Returns a list of coordinates ('x' or 'y') of a LineString geometry"""
    if coord_type == 'x':
        return list( row[geom].coords.xy[0] )
    elif coord_type == 'y':
        return list( row[geom].coords.xy[1] )

# Get x and y coordinates
rails['x'] = rails.apply(getLineCoords, geom='geometry', coord_type='x', axis=1)
rails['y'] = rails.apply(getLineCoords, geom='geometry', coord_type='y', axis=1)

# Drop all the columns except osmid coordinates x and y
rails = rails[["osmid", "x", "y"]]

# Convert to ColumnDataSource
rails = ColumnDataSource(rails)

#### Convert stations to ColumnDataFrame

In [84]:
# Create function getpointcoords
def getPointCoords(row, geom, coord_type):
    """Calculates coordinates ('x' or 'y') of a Point geometry"""
    if coord_type == 'x':
        return row[geom].x
    elif coord_type == 'y':
        return row[geom].y

# Get x and y coordinates
stations['x'] = stations.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)

# Calculate y coordinates
stations['y'] = stations.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)

# Drop all the columns except osmid coordinates x and y
stations = stations.drop('geometry', axis=1)

# Convert to ColumnDataSource
stations = ColumnDataSource(stations)

#### Create Bokeh visualization

In [85]:
# Set outputfile
output_file("bokeh_map.html")

# Get background map provider
#tile_provider = get_provider(Vendors.STAMEN_TERRAIN)
tile_provider = get_provider('CARTODBPOSITRON')

# Convert grid geodataframe to geoJSON for plotting
geosource = GeoJSONDataSource(geojson = stations_population.to_json())

# Create empty figure object
p = figure(title = 'Population xxx ', 
           plot_height = 700 ,
           plot_width = 900, 
           toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset")

# Add background map
p.add_tile(tile_provider)

# Add the rails to the map
p.multi_line('x', 'y', source=rails, color='gray', line_width=1)

# Add the stations to the map
p.circle('x', 'y', source=stations, color='black', size=5)

# ?????
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# Add patch renderer to figure.
grids = p.patches('xs','ys', source = geosource,
                   fill_color = None,
                   line_color = "black", 
                   line_width = 0.25, 
                   fill_alpha = 0.5)

# Create hover tool
p.add_tools(HoverTool(renderers = [grids], 
                      tooltips = [('Station','@Name'),('Type','@Type'),
                                  ('Opened','@Opened'), ('Population','@pop2019')]))

# Show map
show(p)